In [1]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.1                         
Location         D:\Anaconda\envs\tens\lib\site-packages\spacy
Platform         Windows-10-10.0.19045-SP0     
Python version   3.10.8                        
Pipelines        en_core_web_lg (3.3.0), en_core_web_md (3.3.0), en_core_web_sm (3.3.0), en_core_web_trf (3.3.0)



In [1]:
import warnings
warnings.filterwarnings("ignore")
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
import json
f = open('annotations.json' , encoding="utf-8")
TRAIN_DATA = json.load(f)

In [4]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 500/500 [00:00<00:00, 4391.97it/s]


In [6]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     43.80    3.80    3.58    4.04    0.04
  2     200       1205.77   4294.58   87.70   88.89   86.55    0.88
  6     400       3261.04   1194.45   95.71   95.56   95.85    0.96
 11     600       1666.47    574.48   97.58   97.38   97.78    0.98
 16     800        332.76    360.79   98.17   98.47   97.88    0.98
 23    1000        427.02    311.44   99.09   99.29   98.89    0.99
 32    1200        513.50    263.03   99.09   99.19   98.99    0.99
 42    1400        496.81    252.83   99.29   99.39   99.19    0.99
 55    1600        729.93 

[2022-12-30 11:40:26,162] [INFO] Set up nlp object from config
[2022-12-30 11:40:26,172] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-12-30 11:40:26,180] [INFO] Created vocabulary
[2022-12-30 11:40:26,180] [INFO] Finished initializing nlp object
[2022-12-30 11:40:26,431] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [2]:
nlp_ner = spacy.load("./model-best")

In [3]:
t1 = "FINAL PLACE OF DELIVERY * ROBBINSVILLE , NJ DOOR "


doc = nlp_ner(t1)
for ent in doc.ents:
  print(ent.text," ->>> " , ent.label_)

FINAL PLACE OF DELIVERY  ->>>  KEY
ROBBINSVILLE , NJ DOOR  ->>>  VALUE


In [4]:
file1 = open('test.txt', 'r')
Lines = file1.readlines()

c = 0
for line in Lines:
    doc = nlp_ner(line)
    print(line)
    for ent in doc.ents:
        print(ent.text," ->>> " , ent.label_)
    print("\n", "##########################", "\n")
    c = c+1

print(c)

NOTIFY PARTY ( It is agreed that no responsibility shall be attached to the Carrier or its Agents for failure to notify ) OPENROAD TRANSPORTATION KIMBERLY DEVONDE 288 E ELLENDALE AVE DALLAS OR 97338 UNITED STATES OF AMERICA NP > 

NOTIFY PARTY  ->>>  KEY
OPENROAD TRANSPORTATION KIMBERLY DEVONDE 288 E ELLENDALE AVE DALLAS OR 97338 UNITED STATES OF AMERICA NP  ->>>  VALUE

 ########################## 

PRE - CARRIAGE BY HELENA SCHEPERS 009N 

PRE - CARRIAGE BY HELENA SCHEPERS 009N 
  ->>>  KEY

 ########################## 

PLACE OF RECEIPT LEIXOES 

PLACE OF RECEIPT  ->>>  KEY
LEIXOES 
  ->>>  KEY

 ########################## 

OCEAN VESSEL VOYAGE NO . FLAG YM EVOLUTION 055W 

OCEAN VESSEL VOYAGE NO  ->>>  KEY
FLAG YM EVOLUTION 055W 
  ->>>  VALUE

 ########################## 

PORT OF LOADING LEIXOES 

PORT OF LOADING  ->>>  KEY
LEIXOES 
  ->>>  VALUE

 ########################## 

PORT OF DISCHARGE NORFOLK , VA 

PORT OF DISCHARGE  ->>>  KEY
NORFOLK , VA 
  ->>>  VALUE

 #############